In [1]:
#중고차 가격 예측
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv")


display(x_train.head())
display(y_train.head())

,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,13207,hyundi,Santa Fe,2019,Semi-Auto,4223,Diesel,145.0,39.8,2.2
1,17314,vauxhall,GTC,2015,Manual,47870,Diesel,125.0,60.1,2.0
2,12342,audi,RS4,2019,Automatic,5151,Petrol,145.0,29.1,2.9
3,13426,vw,Scirocco,2016,Automatic,20423,Diesel,30.0,57.6,2.0
4,16004,skoda,Scala,2020,Semi-Auto,3569,Petrol,145.0,47.1,1.0


,carID,price
0,13207,31995
1,17314,7700
2,12342,58990
3,13426,12999
4,16004,16990


In [8]:
x_train.drop(columns = {"carID"},inplace=True)

In [35]:
x = x_train.copy()
x.drop(columns = {'brand',"transmission","model","fuelType","year"},inplace=True)
x = x.join(pd.get_dummies(x_train[['brand','transmission','model','fuelType','year']]))
x['year'] = x['year'] - x['year'].min()
x

,mileage,tax,mpg,engineSize,year,brand_audi,brand_bmw,brand_ford,brand_hyundi,brand_merc,...,model_ Z3,model_ Z4,model_ Zafira Tourer,model_ i3,model_ i8,fuelType_Diesel,fuelType_Electric,fuelType_Hybrid,fuelType_Other,fuelType_Petrol
0,4223,145.0,39.8,2.2,22,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,47870,125.0,60.1,2.0,18,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,5151,145.0,29.1,2.9,22,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,20423,30.0,57.6,2.0,19,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,3569,145.0,47.1,1.0,23,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4955,24314,125.0,56.6,2.1,18,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4956,18000,145.0,51.4,3.0,20,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4957,84932,30.0,60.1,2.0,18,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4958,30150,145.0,62.8,3.0,20,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [56]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error,r2_score
import numpy as np 
lin_model = LinearRegression()
kfold = KFold()
acc = []
r2 = []
for train_idx, test_idx in kfold.split(x):
    lin_model.fit(x.loc[train_idx],y_train.loc[train_idx]['price'])
    pred = lin_model.predict(x.loc[test_idx])
    acc.append(mean_absolute_error(pred,y_train.loc[test_idx]['price']))
    r2.append(r2_score(pred,y_train.loc[test_idx]['price']))
print(np.mean(r2))
print(np.mean(acc))

0.8656838861037522
3585.141859423681


In [57]:
from sklearn.ensemble import BaggingRegressor
lin_model = BaggingRegressor()
kfold = KFold()
acc = []
for train_idx, test_idx in kfold.split(x):
    lin_model.fit(x.loc[train_idx],y_train.loc[train_idx]['price'])
    pred = lin_model.predict(x.loc[test_idx])
    acc.append(mean_absolute_error(pred,y_train.loc[test_idx]['price']))
    r2.append(r2_score(pred,y_train.loc[test_idx]['price']))
print(np.mean(r2))
print(np.mean(acc))

0.9077486513267606
2065.6291057572016


In [115]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
minmax = MinMaxScaler()

minmax_df = x.copy()
minmax.fit(x_train[["mileage","tax","mpg","engineSize","year"]])
minmax_df[["mileage","tax","mpg","engineSize","year"]] = minmax.transform(x_train[["mileage","tax","mpg","engineSize","year"]])

lin_model = LinearRegression()
kfold = KFold()
acc = []
r2 = []
for train_idx, test_idx in kfold.split(minmax_df):
    lin_model.fit(minmax_df.loc[train_idx],y_train.loc[train_idx]['price'])
    pred = lin_model.predict(minmax_df.loc[test_idx])
    acc.append(mean_absolute_error(pred,y_train.loc[test_idx]['price']))
    r2.append(r2_score(pred,y_train.loc[test_idx]['price']))
print(np.mean(r2))
print(np.mean(acc))

0.6932884274138745
7534003718.707886


In [113]:
from sklearn.ensemble import BaggingRegressor
lin_model = BaggingRegressor()
kfold = KFold()
acc = []
for train_idx, test_idx in kfold.split(minmax_df):
    lin_model.fit(minmax_df.loc[train_idx],y_train.loc[train_idx]['price'])
    pred = lin_model.predict(minmax_df.loc[test_idx])
    acc.append(mean_absolute_error(pred,y_train.loc[test_idx]['price']))
    r2.append(r2_score(pred,y_train.loc[test_idx]['price']))
print(np.mean(r2))
print(np.mean(acc))

0.82005346338726
2035.1841144546702


In [116]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
std = StandardScaler()

std_df = x.copy()
std.fit(x_train[["mileage","tax","mpg","engineSize","year"]])
std_df[["mileage","tax","mpg","engineSize","year"]] = std.transform(x_train[["mileage","tax","mpg","engineSize","year"]])

lin_model = LinearRegression()
kfold = KFold()
acc = []
r2 = []
for train_idx, test_idx in kfold.split(std_df):
    lin_model.fit(std_df.loc[train_idx],y_train.loc[train_idx]['price'])
    pred = lin_model.predict(std_df.loc[test_idx])
    acc.append(mean_absolute_error(pred,y_train.loc[test_idx]['price']))
    r2.append(r2_score(pred,y_train.loc[test_idx]['price']))
print(np.mean(r2))
print(np.mean(acc))

from sklearn.ensemble import BaggingRegressor
lin_model = BaggingRegressor()
kfold = KFold()
acc = []
for train_idx, test_idx in kfold.split(std_df):
    lin_model.fit(std_df.loc[train_idx],y_train.loc[train_idx]['price'])
    pred = lin_model.predict(std_df.loc[test_idx])
    acc.append(mean_absolute_error(pred,y_train.loc[test_idx]['price']))
    r2.append(r2_score(pred,y_train.loc[test_idx]['price']))
print(np.mean(r2))
print(np.mean(acc))

0.6932725420729516
8223936565.753679
0.8199444390794337
2047.7584026079207


In [73]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
brand_encoder = LabelEncoder()
model_encoder = LabelEncoder()
fuel_encoder = LabelEncoder()
transmission_encoder = LabelEncoder()

brand_encoder.fit(x_train['brand'])
model_encoder.fit(x_train['model'])
fuel_encoder.fit(x_train['fuelType'])
transmission_encoder.fit(x_train['transmission'])

x_train['brand'] = brand_encoder.transform(x_train['brand'])
x_train['model'] = model_encoder.transform(x_train['model'])
x_train['fuelType'] = fuel_encoder.transform(x_train['fuelType'])
x_train['transmission'] = transmission_encoder.transform(x_train['transmission'])

In [91]:
from sklearn.model_selection import GridSearchCV
parameter = {"n_estimators" : [20,50,100], "max_depth": [5,8,10,13,15,20]}
model = RandomForestRegressor()
gcv = GridSearchCV(model,parameter,scoring="r2")
gcv.fit(x_train,y_train['price'])

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_depth': [5, 8, 10, 13, 15, 20],
                         'n_estimators': [20, 50, 100]},
             scoring='r2')

In [92]:
gcv.best_score_

0.9541137506013081

In [99]:
parameter = {"n_estimators" : [20,50,100], "max_depth": [5,8,10,13,15,20]}
model = RandomForestRegressor()
gcv = GridSearchCV(model,parameter,scoring="neg_mean_absolute_error")
gcv.fit(x_train,y_train['price'])

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_depth': [5, 8, 10, 13, 15, 20],
                         'n_estimators': [20, 50, 100]},
             scoring='neg_mean_absolute_error')

In [101]:
print(gcv.best_score_)

-1975.1328302640045
